In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
import os
import pandas as pd

In [2]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 16 * 16, 256)
        self.fc2 = nn.Linear(256, 1)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 16 * 16)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [38]:
# Create training dataset
# Set up the data directories
data_dir = 'Test_Images'
train_dir = os.path.join(data_dir, 'train')

# Define a function to get the labels from the image filenames
def get_label(filename):
    # You can modify this function to extract the label from the filename
    return float(filename.split('.')[0])

# Define a list of dictionaries to store the image filenames and labels
train_data = []

# Iterate over the training images and add them to the list of dictionaries
for filename in os.listdir(train_dir):
    label = get_label(filename)
    train_data.append({'filename': filename, 'label': label})

# Convert the lists of dictionaries to dataframes
train_df = pd.DataFrame(train_data)

# Save the dataframes to CSV files
train_df.to_csv(os.path.join(data_dir, 'train.csv'), index=False)



In [39]:
# Set up the data
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
# train_data = datasets.ImageFolder('Test_Images\train', transform=transform)
dataset = CustomDataset('Test_images/train/train.csv', transform=transform)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
# train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
# labels = torch.tensor([0.5, 0.3, 0.2, ..., 0.8, 0.6, 0.4])  # A tensor of size (1000,)

OSError: [WinError 123] De syntaxis van de bestandsnaam, mapnaam of volumenaam is onjuist: 'Test_Images\train'

In [ ]:
# Define the loss function and optimizer
model = CNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Train the model
for epoch in range(10):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader, 0):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

In [ ]:
# Use the model to predict a value for a new image
test_image = Image.open('path/to/test/image.jpg')
test_image = transform(test_image).unsqueeze(0)
predicted_value = model(test_image).item()
print('Predicted value:', predicted_value)